In [1]:
import os

os.environ['HF_HOME'] = '/home/husein/ssd3'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [26]:
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration, AutoConfig, AutoModelForCausalLM
from transformers import TextStreamer
from datasets import Audio
import torch
import librosa

In [7]:
audio_class = Audio(sampling_rate=16000)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
tokenizer = processor.tokenizer
audio_token = "<|AUDIO|>"
audio_bos_token = "<|audio_bos|>"
audio_eos_token = "<|audio_eos|>"
audio_token_id = processor.tokenizer._convert_token_to_id_with_added_voc('<|AUDIO|>')
pad_token_id = processor.tokenizer.pad_token_id
torch_dtype = torch.bfloat16
min_dtype = torch.finfo(torch_dtype).min
sequence_length = 4096

In [27]:
streamer = TextStreamer(tokenizer)

In [19]:
# !wget https://huggingface.co/malayloraenjoyer/Malaysian-Qwen2-Audio-7B-Instruct-128/resolve/main/checkpoint-800/adapter_model.safetensors

In [10]:
ori_model = Qwen2AudioForConditionalGeneration.from_pretrained(
    'Qwen/Qwen2-Audio-7B-Instruct', torch_dtype = torch.bfloat16,
).cuda()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
state_dict = ori_model.state_dict()

In [14]:
from safetensors import safe_open

f = safe_open(f"adapter_model.safetensors", framework="pt", device=0)
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [16]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A).type(W.dtype)
    B = f.get_tensor(B).type(W.dtype)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|████████████████████████████████████████████████████████████████████████████████████| 226/226 [00:01<00:00, 204.71it/s]


In [29]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "code-website-react.mp3"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

with torch.no_grad():
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True).to('cuda')
    generate_kwargs = dict(
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        streamer=streamer,
        **inputs,
    )
    generation_output = ori_model.generate(**generate_kwargs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Audio 1: <|audio_bos|><|AUDIO|><|audio_eos|>
<|im_end|>
<|im_start|>assistant
Untuk mengkod website dengan React, anda perlu mempunyai beberapa konsep asas dan keupayaan pembangunan web. Berikut adalah langkah-langkah umum untuk mengkodkan sebuah website menggunakan React:

1. **Pengenalan**:
   - Kenal pasti keperluan dan tujuan projek anda. Contohnya, mungkin anda ingin membuat aplikasi berpusat atau aplikasi berprestasi tinggi.

2. **Pemilihan teknologi**:
   - Pilih bahasa pengaturcaraan yang sesuai seperti JavaScript, JSX


In [39]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "speech.mp3"},
        {"type": "text", "text": "transcribe it in whisper word timestamps"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

with torch.no_grad():
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True).to('cuda')
    generate_kwargs = dict(
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        streamer=streamer,
        **inputs,
    )
    generation_output = ori_model.generate(**generate_kwargs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Audio 1: <|audio_bos|><|AUDIO|><|audio_eos|>
transcribe it in whisper word timestamps<|im_end|>
<|im_start|>assistant
<|startoftranscript|><|ms|><|transcribe|><|0.14|> Yes, what can I help you here?<|1.66|><|2.06|> Do you want me to<|3.12|><|3.38|> open Gmail<|3.70|><|3.82|> and Facebook<|4.34|><|4.70|> ya?<|3.98|><|4.72|> Apa yang saya boleh tolong encheuk<|6.44|><|6.82|> nak sahaja buka akaun Gmail dan Facebook?<|8.86|><|7.74|> Ya?<|9.16|><|9.78|> Ya.<|9.84|><|9.90|> No, saya buka akaun<|4.56|><|4.74|> Gmail<|5.78|><|5.84|> dan Facebook<|6.00|><|6.76|> ya.<|7.78|><|7.88|> Ya?<|7.76|><|7.96|> Ya.<|9.88|><|10.08|> Ya.<|9.82|><|10.32|> Kalau tak,<|10.10|><|10.20|> saya buka punya<|10.08|><|10.40|>
